<a href="https://colab.research.google.com/github/3p3r4lt4/3p3r4lt4/blob/main/Ejemplo_aplicativo_Filtrado_basado_en_contenido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

## 1) Lectura de datos

El marco de datos contiene información de diferentes libros, como su autor, editor y título. Usaremos estos datos para crear un sistema de recomendaciones que sugiera lo que un usuario debe leer a continuación en función de sus preferencias de libros actuales
http://www2.informatik.uni-freiburg.de/~cziegler/BX/

In [ ]:
/content/data/BX-CSV-Dump/BX-Books.csv

In [11]:
df = pd.read_csv('/content/data/BX-CSV-Dump/BX-Books.csv',error_bad_lines=False,encoding='latin-1',sep=';')
df.head()

<ipython-input-11-584dbd2a0594>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/data/BX-CSV-Dump/BX-Books.csv',error_bad_lines=False,encoding='latin-1',sep=';')
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping l

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [12]:
df0=df.copy()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


## 2) Preprocesamiento de datos

### 2.1. Eliminar duplicados

In [14]:
#comprobar si hay duplicados
df.duplicated(subset='Book-Title').sum()

29225

In [15]:
#eliminar los duplicados
df = df.drop_duplicates(subset='Book-Title')

In [16]:
#verificamos de nuevo
df.duplicated(subset='Book-Title').sum()

0

### 2.2 Muestreo aleatorio

se muestrea aleatoriamente 15,000 filas del marco de datos para evitar errores de memoria

In [17]:
sample_size = 15000
df = df.sample(n=sample_size, replace=False, random_state=490).reset_index(drop=True)

### 2.3 Procesamiento del Lenguaje Natural

In [18]:
palabras=pd.DataFrame({'Descripción':['Mario escribió la Casa Verde',
                                   'Gabriel escribió Cien Áños de Soledad',
                                   'César escribió Los Herados Negros',
                                   'Mario y Gabriel fueron amigos'
                                  ]})
palabras

,Descripción
0,Mario escribió la Casa Verde
1,Gabriel escribió Cien Áños de Soledad
2,César escribió Los Herados Negros
3,Mario y Gabriel fueron amigos


In [22]:
palabras=pd.DataFrame({'Descripción':['mario escribio Casa Verde',
                                   'gabriel escribio cien años Soledad',
                                   'cesar escribio heraldos negros',
                                   'mario gabriel fueron amigos'
                                  ]})
palabras

,Descripción
0,mario escribio Casa Verde
1,gabriel escribio cien años Soledad
2,cesar escribio heraldos negros
3,mario gabriel fueron amigos


||amigos|años|casa|cesar|cien|escribio|fueron|gabriel|heraldos|mario|negros|soledad|verde|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|0|0|0|1|0|0|1|0|0|0|1|0|0|1|
|1|0|1|0|0|1|1|0|1|0|0|0|1|0|
|2|0|0|0|1|0|1|0|0|1|0|1|0|0|
|3|1|0|0|0|0|0|1|1|0|1|0|0|0|

In [20]:
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.feature_extraction.text import  TfidfVectorizer
contador=CountVectorizer()
vectorizador=TfidfVectorizer()
cantidad=contador.fit_transform(palabras.Descripción)
valores=vectorizador.fit_transform(palabras.Descripción)

In [23]:
print('palalabras')
#print(vectorizador.get_feature_names())
print('cantidad')
print(cantidad)
# print('Puntuaciones')
# print(valores)

palalabras
cantidad
  (0, 9)	1
  (0, 5)	1
  (0, 2)	1
  (0, 12)	1
  (1, 5)	1
  (1, 7)	1
  (1, 4)	1
  (1, 1)	1
  (1, 11)	1
  (2, 5)	1
  (2, 3)	1
  (2, 8)	1
  (2, 10)	1
  (3, 9)	1
  (3, 7)	1
  (3, 6)	1
  (3, 0)	1


#### 2.3.1 Eliminar los espacios en blanco de la columna autor

si un autor se llama James Clear y otro se llama James Patterson, el vectorizador contará la palabra James en ambos casos, y el sistema de recomendación podría considerar que los libros son muy similares, aunque no estén relacionados en absoluto. James Clear escribe libros de autoayuda, mientras que James Patterson es conocido por sus novelas de misterio.

In [24]:
def clean_text(author):
    result = str(author).lower()
    return(result.replace(' ',''))

In [25]:
df['Book-Author'] = df['Book-Author'].apply(clean_text)

In [26]:
df.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0060972491,"Thank You, Jeeves",p.g.wodehouse,1989,Perennial,http://images.amazon.com/images/P/0060972491.0...,http://images.amazon.com/images/P/0060972491.0...,http://images.amazon.com/images/P/0060972491.0...
1,015602781X,Pincher Martin: The Two Deaths of Christopher ...,williamgolding,2002,Harvest Books,http://images.amazon.com/images/P/015602781X.0...,http://images.amazon.com/images/P/015602781X.0...,http://images.amazon.com/images/P/015602781X.0...
2,1561441627,The Nutcracker (Christmas Fun-to-Read Fairy Ta...,robertacollier-morales,1992,Modern Publishing,http://images.amazon.com/images/P/1561441627.0...,http://images.amazon.com/images/P/1561441627.0...,http://images.amazon.com/images/P/1561441627.0...


#### 2.3.2 convertir de mayusculas a ninusculas

In [27]:
df['Book-Title'] = df['Book-Title'].str.lower()
df['Publisher'] = df['Publisher'].str.lower()

In [28]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0060972491,"thank you, jeeves",p.g.wodehouse,1989,perennial,http://images.amazon.com/images/P/0060972491.0...,http://images.amazon.com/images/P/0060972491.0...,http://images.amazon.com/images/P/0060972491.0...
1,015602781X,pincher martin: the two deaths of christopher ...,williamgolding,2002,harvest books,http://images.amazon.com/images/P/015602781X.0...,http://images.amazon.com/images/P/015602781X.0...,http://images.amazon.com/images/P/015602781X.0...
2,1561441627,the nutcracker (christmas fun-to-read fairy ta...,robertacollier-morales,1992,modern publishing,http://images.amazon.com/images/P/1561441627.0...,http://images.amazon.com/images/P/1561441627.0...,http://images.amazon.com/images/P/1561441627.0...
3,1402708610,i know a rhino,charlesfuge,2003,sterling,http://images.amazon.com/images/P/1402708610.0...,http://images.amazon.com/images/P/1402708610.0...,http://images.amazon.com/images/P/1402708610.0...
4,0441586325,norby and the oldest dragon,janetasimov,1993,ace books,http://images.amazon.com/images/P/0441586325.0...,http://images.amazon.com/images/P/0441586325.0...,http://images.amazon.com/images/P/0441586325.0...


#### 2.3.3 juntamos las variables que se va utilizar

In [29]:
# se elimina las variables que no se van utlizar
df2 = df.drop(['ISBN','Image-URL-S','Image-URL-M','Image-URL-L','Year-Of-Publication'],axis=1)

df2['data'] = df2[df2.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(df2['data'].head())

0                     p.g.wodehouse perennial
1                williamgolding harvest books
2    robertacollier-morales modern publishing
3                        charlesfuge sterling
4                       janetasimov ace books
Name: data, dtype: object


#### 2.3.4 Vectorizar el marco de datos

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df2['data'])

La variable "vectorized" es una matriz dispersa con una representación numérica de las cadenas que extrajimos

In [31]:
print(vectorized)

  (0, 15084)	1
  (0, 11320)	1
  (1, 14972)	1
  (1, 5651)	1
  (1, 1595)	1
  (2, 12255)	1
  (2, 10305)	1
  (2, 10241)	1
  (2, 11757)	1
  (3, 2255)	1
  (3, 13587)	1
  (4, 1595)	1
  (4, 6561)	1
  (4, 49)	1
  (5, 1595)	1
  (5, 7130)	1
  (5, 2706)	1
  (5, 1016)	1
  (6, 11757)	1
  (6, 7415)	1
  (6, 12908)	1
  (6, 14490)	1
  (6, 2679)	1
  (7, 1595)	1
  (7, 6465)	1
  :	:
  (14992, 2366)	1
  (14992, 6149)	1
  (14992, 987)	1
  (14993, 1595)	1
  (14993, 10236)	1
  (14993, 11602)	1
  (14993, 4124)	1
  (14994, 5578)	1
  (14994, 14025)	1
  (14995, 6125)	1
  (14995, 10602)	1
  (14995, 11759)	1
  (14995, 5566)	1
  (14995, 13767)	1
  (14996, 11697)	1
  (14996, 7859)	1
  (14996, 10884)	1
  (14997, 11747)	1
  (14997, 14372)	1
  (14997, 4605)	1
  (14997, 4606)	1
  (14998, 5568)	1
  (14998, 12506)	1
  (14999, 5600)	1
  (14999, 1376)	1


## 3) Construyendo el sistema de recomendación

![image.png](attachment:image.png)

![image.png](attachment:image.png)

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(vectorized)

In [33]:
similarities

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [34]:
df = pd.DataFrame(similarities, columns=df['Book-Title'], index=df['Book-Title']).reset_index()
df.head()

Book-Title,Book-Title,"thank you, jeeves",pincher martin: the two deaths of christopher martin,the nutcracker (christmas fun-to-read fairy tales ser),i know a rhino,norby and the oldest dragon,his eminence and hizzoner,el oro de los sueã±os (leer en espaã±ol nivel 4),celestine insights,feng shui and money: a nine-week program for creating wealth using ancient principles and techniques,...,"the santa suit (harlequin american romance, no. 708)",where the gods reign: plants and peoples of the colombian amazon,the terrible troll-bird,virginia embrace (magnolia road),"take 5, volume 4 (under the knife, adam's gift, return to yesterday, everything but time, marriage diamond style)",taking the anxiety out of taking tests: a step-by-step guide,encyclopedia of women's health issues,fodor's 2002 new york city (fodor's new york city),"c. s. lewis: a biography,revised edition",the late night muse: a novel
0,"thank you, jeeves",1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,pincher martin: the two deaths of christopher ...,0.0,1.000000,0.0,0.0,0.333333,0.288675,0.000000,0.288675,0.0,...,0.0,0.0,0.235702,0.288675,0.0,0.0,0.0,0.0,0.0,0.0
2,the nutcracker (christmas fun-to-read fairy ta...,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.223607,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,i know a rhino,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,norby and the oldest dragon,0.0,0.333333,0.0,0.0,1.000000,0.288675,0.000000,0.288675,0.0,...,0.0,0.0,0.235702,0.288675,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df['Book-Title'].unique()[500]

'a gift of sanctuary (an owen archer mystery)'

In [36]:
input_book = df['Book-Title'].unique()[500]
input_book

'a gift of sanctuary (an owen archer mystery)'

In [37]:
#input_book = 'far beyond the stars (star trek deep space nine)'
recommendations = pd.DataFrame(df.nlargest(5,input_book)['Book-Title'])
recommendations = recommendations[recommendations['Book-Title']!=input_book]
print(recommendations)

                                             Book-Title
1839  an intimate ghost : a jane lawless mystery (ja...
1852  the doctor makes a dollhouse call (dr. fenimor...
2302  the skeleton in the closet (st. martin's minot...
2860  a mist of prophecies : a novel of ancient rome...
